# Engenharia de Atributos

## Importação

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
dataset = pd.read_csv('credit_simple.csv', sep=";")

In [4]:
dataset.shape

(1000, 8)

In [8]:
dataset.dtypes

SALDO_ATUAL         float64
RESIDENCIADESDE       int64
IDADE                 int64
OUTROSPLANOSPGTO     object
DATA                 object
ESTADOCIVIL          object
PROPOSITO            object
CLASSE               object
dtype: object

In [5]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


## Separação das variáveis

In [6]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1]

## Tratamento e Limpeza dos Dados

### Valores Faltantes, Outliers e Transformações

- Valores faltantes

In [7]:
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

Para o 'SALDO_ATUAL' os valores serão preenchidos com a mediana.

In [9]:
mediana = X['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [10]:
# Preenchendo os valores faltantes com a mediana
X['SALDO_ATUAL'].fillna(mediana, inplace=True)

C:\Users\Emanuelle\AppData\Local\Temp\ipykernel_25832\1818155535.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana, inplace=True)


In [11]:
# Verificando
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

Para valores categóricos como em 'ESTADOCIVIL', usa-se a moda para preencher os valores.

In [12]:
# Buscando o valor que mais se repete
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [13]:
# Preenchendo com o valor que mais se repete
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

- Outliers

In [14]:
# Calculando desvio padrão para verficiar outliers
desv = X['SALDO_ATUAL'].std()
desv

np.float64(685936688.9820067)

In [15]:
# Verificando outliers
X.loc[X['SALDO_ATUAL'] >= 2 * desv, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [16]:
# Preenchendo os valores dos outliers com a mediana
X.loc[X['SALDO_ATUAL'] >= 2 * desv, 'SALDO_ATUAL'] = mediana

In [17]:
# Verificando
X.loc[X['SALDO_ATUAL'] >= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


- Erro de digitação e agrupamento de valores pequenos.

In [18]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [19]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'

In [20]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

- Extração de características

In [21]:
X['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [22]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [23]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [25]:
# Criando novos atributos
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIA'] = X['DATA'].dt.day_name()

In [26]:
X['DIA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIA, Length: 1000, dtype: object

### Fazendo o Encoder dos Atributos

 - Transformando valores categóricos em numéricos com Label Encoder

In [27]:
labelEncoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelEncoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelEncoder1.fit_transform(X['PROPOSITO'])
X['DIA'] = labelEncoder1.fit_transform(X['DIA'])

In [28]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


 - Transformando valores categóricos em binário com One Hot Encoder. Vale lembrar que os atributos nesse caso viram uma coluna.

In [30]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [31]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')

In [32]:
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


### Padronização dos valores

 - Normalização dos dados

In [33]:
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [34]:
X = pd.concat([X, z, pd.DataFrame(m,columns=['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N'])], axis=1)

In [36]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,False,True,False,0.572709,1.046987,0.898982


### Dropando Colunas

In [37]:
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)

In [38]:
X.head()

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,7,2019,1,5,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,True,False,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982
